
----

In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

%run ../load_magic/storage.py
s = Storage()
[f's.{fn}' for fn in dir(s) if not fn.startswith('_')]

['s.attempt_to_pickle', 's.data_csv_folder', 's.data_folder', 's.encoding_type', 's.load_csv', 's.load_dataframes', 's.load_object', 's.pickle_exists', 's.save_dataframes', 's.saves_csv_folder', 's.saves_folder', 's.saves_pickle_folder', 's.store_objects']

In [109]:

import random

type_suffix_list = ['dict', 'df', 'list', 'DICT', 'LIST']
dict_suffix_list = ['dict', 'DICT']
for file_name in os.listdir(s.saves_pickle_folder):
    if file_name.endswith('.pickle'):
        obj_name = file_name.split('.')[0]
        type_suffix = obj_name.split('_')[-1]
        if type_suffix in dict_suffix_list:
            column_name = obj_name[:-5].lower()
            exec_str = f"{obj_name} = s.load_object('{obj_name}')"
            eval_str = f"s.load_object('{obj_name}')"
            dict_obj = eval(eval_str)
            keys_list = list(dict_obj.keys())
            idx = random.randint(0, len(keys_list))
            print(f'{column_name}: {keys_list[idx][:50]}...,  {keys_list[idx+1][:50]}..., etc.')

basic_tags: thing for our employees, because we know that thei...,  leads us to great places. Life. Happiness. Innovat..., etc.
child_strs_list: Sr._Data_Analyst_-_Redmond,_WA_-_Indeed.com_a3bcc3...,  Sr._Data_Scientist-_Machine_Learning_AI_REMOTE_-_S..., etc.
description: training_duration...,  inference_duration..., etc.
fit_estimators: BaggingClassifier...,  ExtraTreesClassifier..., etc.
header_pattern: AVP,_Data_Science_-_Fort_Mill,_SC_29715_-_Indeed.c...,  Big_Data_Engineer_-_New_York,_NY_10001_-_Indeed.co..., etc.
navigable_parent_is_header: <li>Bonus pay</li>...,  <p>Education:</p>..., etc.
navigable_parent_is_qual: <b>Benefits</b>...,  <li>Competitive salary and equity</li>..., etc.
pos_explanation: H-JD...,  H-SP..., etc.
remaining_quals_child_strs_list: Data_Scientist_-_Plano,_TX_-_Indeed.com_4c8feeee8e...,  Data_Scientist_-_Princeton,_NJ_08540_-_Indeed.com_..., etc.


In [3]:

NAVIGABLE_PARENT_IS_HEADER_DICT = s.load_object('NAVIGABLE_PARENT_IS_HEADER_DICT')
max(len(child_str) for child_str in NAVIGABLE_PARENT_IS_HEADER_DICT.keys())*2

2224

In [3]:

%run ../py/html_analysis.py
ha = HeaderAnalysis()

In [16]:

def get_execution_results(sql_str, database='Jobhunting'):
    if database == 'Jobhunting':
        conn, cursor = get_jh_conn_cursor()
    else:
        conn = pyodbc.connect(
            driver='{SQL Server}',
            server='localhost\MSSQLSERVER01',
            database=database,
            trusted_connection=True
        )
        cursor = conn.cursor()
    #print(sql_str)
    cursor.execute(sql_str)
    try:
        row_objs_list = cursor.fetchall()
        row_tuples_list = [tuple(row_obj) for row_obj in row_objs_list]
    except:
        row_tuples_list = []
    conn.commit()
    cursor.close()
    
    return row_tuples_list

In [17]:

def get_jh_conn_cursor():
    conn = pyodbc.connect(
        driver='{SQL Server}',
        server='localhost\MSSQLSERVER01',
        database='Jobhunting',
        trusted_connection=True
    )
    cursor = conn.cursor()
    
    return conn, cursor

In [18]:

def create_navigable_parent_sequence_table_dataframe():
    files_list = os.listdir(ha.SAVES_HTML_FOLDER)
    rows_list = []
    for file_name in files_list:
        child_strs_list = ha.get_child_strs_from_file(file_name)
        for i, child_str in enumerate(child_strs_list):
            row_dict = {}
            row_dict['file_name'] = file_name
            row_dict['navigable_parent'] = child_str
            row_dict['sequence_order'] = i
            rows_list.append(row_dict)
    navigable_parent_sequence_table_df = pd.DataFrame(rows_list)
    navigable_parent_sequence_table_df.index.name = 'navigable_parent_sequence_id'
    #s.save_dataframes(include_index=True, navigable_parent_sequence_table_df=navigable_parent_sequence_table_df)
    
    return navigable_parent_sequence_table_df

In [27]:

import pyodbc

create_database_sql_str = '''
IF (EXISTS (
    SELECT * FROM master.dbo.sysdatabases
    WHERE name = N'Jobhunting'
    ))
    BEGIN
        SELECT 'DATABASE already EXISTS' AS Message
    END
ELSE
    BEGIN
        CREATE DATABASE [Jobhunting];
        SELECT * FROM master.dbo.sysdatabases
        WHERE name = N'Jobhunting'
    END;'''
create_table_sql_str = '''
IF (EXISTS (
    SELECT * FROM INFORMATION_SCHEMA.TABLES
    WHERE TABLE_NAME = N'NavigableParentSequence'
    ))
    BEGIN
        SELECT 'TABLE_NAME already EXISTS' AS Message
    END
ELSE
    BEGIN
        CREATE TABLE NavigableParentSequence(
            navigable_parent_sequence_id INT PRIMARY KEY,
            file_name VARCHAR (255) NOT NULL,
            navigable_parent NVARCHAR (2224) NOT NULL,
            sequence_order INT NOT NULL
        );
        SELECT * FROM INFORMATION_SCHEMA.TABLES
        WHERE TABLE_NAME = N'NavigableParentSequence'
    END;'''
insert_columns_sql_str = '''
SET QUOTED_IDENTIFIER OFF
SET ANSI_NULLS ON
IF (EXISTS (
    SELECT * FROM NavigableParentSequence WHERE
        file_name = ? AND
        sequence_order = ?
    ))
    BEGIN
        SELECT 'Row already EXISTS' AS Message
    END
ELSE
    BEGIN
        INSERT INTO NavigableParentSequence (
            navigable_parent_sequence_id,
            file_name,
            navigable_parent,
            sequence_order
        ) values(
            ?,
            ?,
            ?,
            ?
        );
        SELECT * FROM NavigableParentSequence WHERE
            file_name = ? AND
            sequence_order = ?
    END;'''

In [30]:

# https://stackoverflow.com/questions/5160742/how-to-store-and-search-for-a-sequence-in-a-rdbms
def create_navigable_parent_sequence_table():
    row_tuples_list = get_execution_results(create_database_sql_str, database='master')
    row_tuples_list = get_execution_results(create_table_sql_str)
    
    # Create the navigable html strings dataset
    navigable_parent_sequence_table_df = create_navigable_parent_sequence_table_dataframe()
    
    # Insert Dataframe into SQL Server:
    conn, cursor = get_jh_conn_cursor()
    for row_index, row_series in navigable_parent_sequence_table_df.iterrows():
        #print(insert_columns_sql_str)
        try:
            cursor.execute(insert_columns_sql_str, row_series.file_name, row_series.sequence_order, row_index, row_series.file_name,
                           row_series.navigable_parent, row_series.sequence_order, row_series.file_name, row_series.sequence_order)
        except Exception as e:
            print(str(e).strip())
            print(insert_columns_sql_str.replace('?', '"{}"').format(row_series.file_name, row_series.sequence_order, row_index,
                                                                     row_series.file_name, row_series.navigable_parent, row_series.sequence_order,
                                                                     row_series.file_name, row_series.sequence_order))
            break
        try:
            row_objs_list = cursor.fetchall()
        except:
            pass
    conn.commit()
    cursor.close()

In [31]:

create_navigable_parent_sequence_table()


----
# Explore existing columns on the server

In [10]:

# sqlcmd -S DAVEZ-ASUS-LAPT\MSSQLSERVER01 -E
connection_str = 'Server=localhost\MSSQLSERVER01;Database=master;Trusted_Connection=True;'

In [30]:

table_cat_idx = 0
table_schem_idx = 1
table_name_idx = 2
column_name_idx = 3
data_type_idx = 4
type_name_idx = 5
column_size_idx = 6
buffer_length_idx = 7
decimal_digits_idx = 8
num_prec_radix_idx = 9
nullable_idx = 10
remarks_idx = 11
column_def_idx = 12
sql_data_type_idx = 13
sql_datetime_sub_idx = 14
char_octet_length_idx = 15
ordinal_position_idx = 16
is_nullable_idx = 17

In [63]:

conn = pyodbc.connect(
    driver='{SQL Server}',
    server='localhost\MSSQLSERVER01',
    database='master',
    trusted_connection=True
)
cursor = conn.cursor()
cursor.execute('SELECT name FROM master.dbo.sysdatabases;')

databases_list = []
for database_row_obj in cursor:
    database_row_tuple = tuple(database_row_obj)
    database_name = database_row_tuple[0]
    databases_list.append(database_name)
conn.close()
databases_list

['master', 'tempdb', 'model', 'msdb', 'Jobhunting']

In [86]:

rows_list = []
for database_name in databases_list:
    database_conn = pyodbc.connect(
        driver='{SQL Server}',
        server='localhost\MSSQLSERVER01',
        database=database_name,
        trusted_connection=True
    )
    database_cursor = database_conn.cursor()
    sql_str = f'SELECT * FROM {database_name}.INFORMATION_SCHEMA.TABLES;'
    #print(sql_str)
    database_cursor.execute(sql_str)
    table_row_objs_list = database_cursor.fetchall()
    for table_row_obj in table_row_objs_list:
        table_row_tuple = tuple(table_row_obj)
        if len(table_row_tuple) >= 3:
            row_dict = {}
            row_dict['database_name'] = database_name
            table_name = table_row_tuple[2]
            row_dict['table_name'] = table_name
            rows_list.append(row_dict.copy())
    database_conn.close()
column_names_df = pd.DataFrame(rows_list)

In [87]:

column_names_df.sample(4).T

,91,71,150,89
database_name,msdb,msdb,msdb,msdb
table_name,syspolicy_object_sets,syscollector_collection_sets_internal,syspolicy_policies,sysmail_server


In [97]:

rows_list = []
for row_index, row_series in column_names_df.iterrows():
    database_name = row_series.database_name
    table_name = row_series.table_name
    database_conn = pyodbc.connect(
        driver='{SQL Server}',
        server='localhost\MSSQLSERVER01',
        database=database_name,
        trusted_connection=True
    )
    database_cursor = database_conn.cursor()
    column_row_objs_list = database_cursor.columns(table=table_name)
    for column_row_obj in column_row_objs_list:
        column_row_tuple = tuple(column_row_obj)
        row_dict = {}
        row_dict['database_name'] = database_name
        row_dict['table_name'] = column_row_tuple[table_name_idx]
        row_dict['column_name'] = column_row_tuple[column_name_idx]
        row_dict['table_cat'] = column_row_tuple[table_cat_idx]
        row_dict['table_schem'] = column_row_tuple[table_schem_idx]
        row_dict['data_type'] = column_row_tuple[data_type_idx]
        row_dict['type_name'] = column_row_tuple[type_name_idx]
        row_dict['column_size'] = column_row_tuple[column_size_idx]
        row_dict['buffer_length'] = column_row_tuple[buffer_length_idx]
        row_dict['decimal_digits'] = column_row_tuple[decimal_digits_idx]
        row_dict['num_prec_radix'] = column_row_tuple[num_prec_radix_idx]
        row_dict['nullable'] = column_row_tuple[nullable_idx]
        row_dict['remarks'] = column_row_tuple[remarks_idx]
        row_dict['column_def'] = column_row_tuple[column_def_idx]
        row_dict['sql_data_type'] = column_row_tuple[sql_data_type_idx]
        row_dict['sql_datetime_sub'] = column_row_tuple[sql_datetime_sub_idx]
        row_dict['char_octet_length'] = column_row_tuple[char_octet_length_idx]
        row_dict['ordinal_position'] = column_row_tuple[ordinal_position_idx]
        row_dict['is_nullable'] = column_row_tuple[is_nullable_idx]
        rows_list.append(row_dict.copy())
    database_conn.close()
column_names_df = pd.DataFrame(rows_list)

In [98]:

column_names_df.sample(5).T

,1744,1985,1991,1929,711
database_name,msdb,msdb,msdb,msdb,msdb
table_name,backupset,sysutility_ucp_mi_database_health_internal,restorefile,backupfile,sysutility_ucp_mi_health
column_name,last_recovery_fork_guid,over_utilized_count,destination_phys_drive,filegroup_guid,mi_processor_health_state
table_cat,msdb,msdb,msdb,msdb,msdb
table_schem,dbo,dbo,dbo,dbo,dbo
data_type,-11,4,-9,-11,4
type_name,uniqueidentifier,int,nvarchar,uniqueidentifier,int
column_size,36,10,260,36,10
buffer_length,16,4,520,16,4
decimal_digits,NaN,0,NaN,NaN,0


In [22]:

row_obj.cursor_description

(('TABLE_CATALOG', str, None, 128, 128, 0, True),
 ('TABLE_SCHEMA', str, None, 128, 128, 0, True),
 ('TABLE_NAME', str, None, 128, 128, 0, False),
 ('TABLE_TYPE', str, None, 10, 10, 0, True))


----

In [116]:

import sys

command_str = f'{sys.executable} -m pip install C:\\Users\\dev\\Downloads\\pymssql-2.1.5-cp39-cp39-win_amd64.whl'
print(command_str)
!{command_str}

C:\Users\dev\Anaconda3\envs\jh\python.exe -m pip install C:\Users\dev\Downloads\pymssql-2.1.5-cp39-cp39-win_amd64.whl
Processing c:\users\dev\downloads\pymssql-2.1.5-cp39-cp39-win_amd64.whl


In [112]:

import sys

command_str = f'{sys.executable} -m pip install --upgrade pymssql'
print(command_str)
!{command_str}

C:\Users\dev\Anaconda3\envs\jh\python.exe -m pip install --upgrade pymssql
  Using cached pymssql-2.1.5.tar.gz (167 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\dev\Anaconda3\envs\jh\python.exe' 'C:\Users\dev\Anaconda3\envs\jh\lib\site-packages\pip\_vendor\pep517\_in_process.py' build_wheel 'C:\Users\dev\AppData\Local\Temp\tmpblqdrc6c'
       cwd: C:\Users\dev\AppData\Local\Temp\pip-install-4a2_b6cz\pymssql
  Complete output (16 lines):
  setup.py: platform.system() => 'Windows'
  setup.py: platform.architecture() => ('64bit', 'WindowsPE')
  running bdist_wheel
  running build
  running build_ext
  cythoning src\_mssql.pyx to src\_mssql.c
  cythoning src\pymssql.pyx to src\pymssql.c
  building '_mssql' extension
  creating build
  creating build\temp.win-amd64-3.9
  creating build\temp.win-amd64-3.9\Release
  creating build\temp.win-amd64-3.9\Release\src
  C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC\Tools\MSVC\14.23.28105\bin\HostX86\x64\cl.exe /c /nologo /Ox /W3 /GL /DNDEBUG /MD -Ifreetds\include -IC:\Users\dev\AppData\Local\Temp\pip-install-4a2_b6cz\p

Failed to build pymssql
